# Imports

In [1]:
import os
import pathlib
import random
from math import sqrt
import ujson

import pandas as pd
import plotly.express as px

if "PATH" not in globals():
    PATH = pathlib.Path.cwd() / "src"

# set current working directory to the path
os.chdir(PATH)
print(f"Current path: {PATH}")

Current path: /home/obergam/code/cymoo/src


In [2]:
# update the PYTHONPATH
!export PYTHONPATH=$PYTHONPATH:{PATH}

In [3]:
from src.MOO import MOO
from src.optimizers.NSGA2 import NSGA2
from src.optimizers.NSRA import NSRA
from src.optimizers.NSWGE import NSWGE
from src.problems.multitask_routing.Network import Network
from src.problems.multitask_routing.Task import Task
from src.problems.Solution import Solution

# Defining the parameters

In [4]:
random.seed(10)

## Layers

In [5]:
# PROBLEM DEFINITION
paramsLayerOne = {
    "unit": {
        "tag": "DEVICE",
        "computingSpeed": lambda: random.randint(5, 10),
        "positionX": 0,
        "positionY": 0,
        "throughput": 0.5,
        "pollution": lambda: round(random.random(), 2),
        "cost": lambda: round(random.random(), 2),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 1,
        "flowRate": lambda: 1,
    },
    "numberNewUnits": 10,
}

paramsLayerTwo = {
    "unit": {
        "tag": "FOG",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(5, 10),
        "positionX": lambda x: x.positionX + random.randint(-2, 2),
        "positionY": lambda x: x.positionY + random.randint(-2, 2),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 2
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 20,
}

paramsLayerThree = {
    "unit": {
        "tag": "CLOUD",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(50, 100),
        "positionX": lambda x: x.positionX + random.randint(-5, 5),
        "positionY": lambda x: x.positionY + random.randint(-5, 5),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 3
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 10,
}

## Tasks

In [6]:
tasks = tuple(
    Task(random.randint(100, 1000000), random.randint(100, 1000000)) for _ in range(5)
)

## Network

In [7]:
optimUnits = {
    "processingTime": "Processing time (s)",
    "cost": "Cost (€)",
    "pollution": "Pollution (g)",
}
optimDirections = {
    "processingTime": "min",
    "cost": "min",
    "pollution": "min",
}
problem = Network(
    "DEVICE",
    tasks=tasks,
    optimDirections=optimDirections,
    minDepth=1,
    maxDepth=15,
    mutationRate=0.1,
    layers=[paramsLayerOne, paramsLayerTwo, paramsLayerThree],
)

## Optimization

In [8]:
nIterations = 100
nSolutions = 1000

# Optimization

## Training

In [9]:
moo = MOO(problem)

nsga2_paretos = moo.optimize(
    NSGA2,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    ratioKept=0.5,
)
nsra_paretos = moo.optimize(
    NSRA,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    ratioKept=0.5,
)
nswge_paretos = moo.optimize(
    NSWGE,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
)

Optimizing with NSGA2...


100%|██████████| 100/100 [01:21<00:00,  1.23it/s]


Displaying pareto solutions...
{'processingTime': 1125291.888888889, 'cost': 550930.9323031553, 'pollution': 1484628.8037291132}
{'processingTime': 1125291.888888889, 'cost': 645603.011498018, 'pollution': 707765.2179941166}
{'processingTime': 1115814.9, 'cost': 1267990.2259579017, 'pollution': 921763.9088428498}
{'processingTime': 1115814.9, 'cost': 1386295.2232097294, 'pollution': 507312.5040176779}
{'processingTime': 1125291.888888889, 'cost': 1032708.5844868274, 'pollution': 545326.5139732591}

Optimizing with NSRA...


100%|██████████| 100/100 [01:27<00:00,  1.15it/s]


Displaying pareto solutions...
{'processingTime': 4264242.833333333, 'cost': 2850520.678293016, 'pollution': 4751997.16595265}
{'processingTime': 2828350.693543404, 'cost': 6599550.806535725, 'pollution': 4390240.695442673}
{'processingTime': 2691382.8988285214, 'cost': 2932969.278463312, 'pollution': 4416046.786181757}
{'processingTime': 4243935.0, 'cost': 5788356.390973598, 'pollution': 4338152.308967329}

Optimizing with NSWGE...


100%|██████████| 100/100 [16:53<00:00, 10.13s/it]


Displaying pareto solutions...
{'processingTime': 2736154.4706358784, 'cost': 2004025.1263651669, 'pollution': 3261040.314340026}
{'processingTime': 2628664.190108381, 'cost': 2288206.15081599, 'pollution': 2216976.1887119226}



## Evaluation

In [10]:
MOO.relative_efficiency(nsra_paretos, nsga2_paretos, Solution.optimDirections, verbose=True)
MOO.relative_efficiency(nswge_paretos, nsga2_paretos, Solution.optimDirections, verbose=True)
MOO.relative_efficiency(nswge_paretos, nsra_paretos, Solution.optimDirections, verbose=True)


Relative efficiency: 0 % of <class 'src.optimizers.NSRA.NSRA'> solutions are undominated by <class 'src.optimizers.NSGA2.NSGA2'> solutions
Relative efficiency: 100 % of <class 'src.optimizers.NSGA2.NSGA2'> solutions are undominated by <class 'src.optimizers.NSRA.NSRA'> solutions
Relative efficiency: 0 % of <class 'src.optimizers.NSWGE.NSWGE'> solutions are undominated by <class 'src.optimizers.NSGA2.NSGA2'> solutions
Relative efficiency: 100 % of <class 'src.optimizers.NSGA2.NSGA2'> solutions are undominated by <class 'src.optimizers.NSWGE.NSWGE'> solutions
Relative efficiency: 100 % of <class 'src.optimizers.NSWGE.NSWGE'> solutions are undominated by <class 'src.optimizers.NSRA.NSRA'> solutions
Relative efficiency: 0 % of <class 'src.optimizers.NSRA.NSRA'> solutions are undominated by <class 'src.optimizers.NSWGE.NSWGE'> solutions


(1, 0)

# Results

In [11]:
def generate_violin_plot(df, title, yaxis_title="Value"):
    fig = px.violin(
        df,
        x="Epoch",
        y="Value",
        box=True,
        points=False,
        hover_data=df.columns,
        template="plotly_white",
        color_discrete_sequence=["#1f77b4"],
    )
    fig.update_layout(
        title=title,
        xaxis_title="Epoch",
        yaxis_title=yaxis_title,
        font=dict(
            family="Times New Roman",
            size=12,
        ),
    )
    fig.show()

In [12]:
# load pareto data from json
for file in os.listdir(PATH.parent / "paretos"):
    if file.endswith(".json"):
        # load file as dict
        print(file)
        with open(PATH.parent / "paretos" / file, "r") as f:
            data = ujson.load(f)
        for objective in optimDirections.keys():
            # unstack values
            df = (
                pd.DataFrame.from_dict(data[objective], orient="index")
                .stack()
                .reset_index()
                .rename(columns={"level_0": "Epoch", 0: "Value"})
                .drop(columns="level_1")
            )
            # generate plot
            generate_violin_plot(
                df,
                title=f"Violin plot of the pareto solutions for each epoch (10 epochs) for the {objective} objective",
                yaxis_title=optimUnits[objective],
            )


NSRA.json


NSWGE.json


NSGA2.json
